In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

from typing import *  # pyright: ignore

import numpy as np
from numpy.typing import NDArray, ArrayLike, DTypeLike

In [6]:
import metpy.units
import metpy.calc

import pint
from pint.registry import Quantity
from pint.facets.plain import PlainQuantity
from pint.facets.plain.quantity import MagnitudeT, Scalar

if TYPE_CHECKING:
    from literal_unit import LiteralUnit
else:
    LiteralUnit = str

In [117]:
import pint.registry


ureg = pint.UnitRegistry()


def unit(x: Any):
    return ureg(x).units


# ureg = pint.registry.GenericUnitRegistry()
ureg.default_format = ".3f~P"

# • [time]
s = unit("second")
min_ = unit("minute")
hr = unit("hour")


# • [temperature]
K = unit("kelvin")
C = unit("celsius")
F = unit("fahrenheit")


# • [length]
m = unit("meter")
km = unit("kilometer")

in_ = unit("inch")
ft = unit("foot")
mi = unit("mile")


# • pressure
Pa = unit("pascal")
hPa = unit("hectopascal")
kPa = unit("kilopascal")
mbar = unit("millibar")

# • energy
J = unit("joule")
kJ = unit("kilojoule")
cal = unit("calorie")
kcal = unit("kilocalorie")
mol = unit("mole")

# - [mass]
kg = unit("kilogram")


# - angle
deg = unit("degree")
rad = unit("radian")


dimensionless = unit("dimensionless")
# - speed
mps = m / s
kph = km / hr
mph = mi / hr
kts = unit("knot")

# - image processing
px = unit("pixel")
dpi = unit("dot/inch")
ppi = unit("pixel/inch")

In [115]:
# type(m.units / s.units**2)

# 9.80665 * m.units / s.units**2

9.80665 <Unit('meter / second ** 2')>

In [118]:
def unit(x: Any):
    return ureg(x).units


def isscaler(x: Any) -> TypeGuard[Scalar]:
    return np.isscalar(x.magnitude if isinstance(x, Quantity) else x)


def quantity(x: MagnitudeT, unit: pint.Unit | Quantity | LiteralUnit = dimensionless) -> PlainQuantity[MagnitudeT]:
    unit = ureg(unit) if isinstance(unit, str) else unit
    return (x if isscaler(x) else np.asanyarray(x)) * unit  # type: ignore


quantity(9.80665, m / s**2)

9.80665 <Unit('meter / second ** 2')>

In [73]:

EARTH_GRAVITY = g = quantity(9.80665, m / s**2)
""">>> EARTH_GRAVITY `little g`"""
GRAVITATIONAL_CONSTANT = G = quantity(6.67408e-11, m**3 / kg / s**2)
"""`big G`"""""
EARTH_RADIUS = Re = quantity(6371008.7714, m)
GEOCENTRIC_GRAVITATIONAL_CONSTANT = GM = quantity(3986005e8, m**3 / s**2)
EARTH_MASS = Me = GEOCENTRIC_GRAVITATIONAL_CONSTANT / GRAVITATIONAL_CONSTANT


ABSOLUTE_ZERO = K0 = quantity(-273.15, K)
MOLAR_GAS_CONSTANT = R = quantity(8.314462618, J / mol / K)


STANDARD_TEMPERATURE = t0 = 288.0 * K
"""Standard temperature at sea level."""
p0 = 1013.25 * hPa  # type: PlainQuantity[float]
"""Standard pressure at sea level."""
LAPSE_RATE = gamma = 6.5 * K / km
"""Standard lapse rate."""

MOLAR_GAS_CONSTANT = R = quantity(8.314462618, J / mol / K)
DRY_AIR_MOLECULAR_WEIGHT_RATIO = Md = quantity(28.96546e-3, kg / mol)

# Dry air

DRY_AIR_GAS_CONST = Rd = R / Md
# Md = dry_air_molecular_weight = units.Quantity(28.96546e-3, 'kg / mol')
# Rd = dry_air_gas_constant = R / Md
dry_air_spec_heat_ratio = 1.4 * dimensionless
Cp_d = dry_air_spec_heat_press = dry_air_spec_heat_ratio * Rd / (dry_air_spec_heat_ratio - 1)
Cv_d = dry_air_spec_heat_vol = Cp_d / dry_air_spec_heat_ratio
P0 = pot_temp_ref_press = 1000.0 * mbar
rho_d = dry_air_density_stp = (P0 / (Rd * -K0)).to("kg / m^3")
# Dry air
DRY_AIR_MOLECULAR_WEIGHT_RATIO = Md = 28.96546e-3 * kg / mol
DRY_AIR_GAS_CONST = Rd = R / Md
# Md = dry_air_molecular_weight = units.Quantity(28.96546e-3, 'kg / mol')
# Rd = dry_air_gas_constant = R / Md
dry_air_spec_heat_ratio = 1.4 * dimensionless
Cp_d = dry_air_spec_heat_press = dry_air_spec_heat_ratio * Rd / (dry_air_spec_heat_ratio - 1)
Cv_d = dry_air_spec_heat_vol = Cp_d / dry_air_spec_heat_ratio
P0 = pot_temp_ref_press = 1000.0 * mbar
rho_d = dry_air_density_stp = (P0 / (Rd * -K0)).to("kg / m^3")

In [101]:
def pressure2height(pressure: PlainQuantity[float | int]) -> PlainQuantity[float]:
    return t0 / gamma * (1 - (pressure / p0) ** (Rd * gamma / g))


def height2pressure(height: PlainQuantity[float | int] | NDArray) -> PlainQuantity[float]:
    return p0 * (1 - (gamma / t0) * height) ** (g / (Rd * gamma))


levels = [850.0, 700.0, 500.0, 300.0]
assert np.all(pressure2height(levels * hPa) == metpy.calc.pressure_to_height_std(levels * metpy.units.units.hPa))

heights = pressure2height(levels * hPa)
assert np.all(
    metpy.calc.height_to_pressure_std(metpy.calc.pressure_to_height_std(levels * metpy.units.units.hPa))
    == height2pressure(pressure2height(levels * hPa))
)